In [2]:
# Install BigQuery client library (run this cell once)
%pip install google-cloud-bigquery --quiet
%pip install bigquery-magics --quiet
%pip install db-dtypes --quiet
# Import libraries
from google.cloud import bigquery
import pandas as pd
%load_ext bigquery_magics
import db_dtypes
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "salesforce-465614-2cf9e37da64b.json"
from google.cloud import bigquery
# Initialize BigQuery client
client = bigquery.Client()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [11]:
%%bqsql
select * from salesforce-465614.cust_analytics.report_transactions
limit 2

Query is running:   0%|          |

Downloading:   0%|          |

,transaction_id,customer_id,product_id,Store_id,order_status,Purchase_Date,category,sub_category,Unit_Price,quantity,Total_Price,payment_method,channel,loyalty_program,age,gender,city
0,T509183,C464,P845,STORE_16,Completed,2024-06-04,Beauty,Haircare,927.10,5,4635.50,PayPal,Physical,true,45,Male,Williamburgh
1,T539986,C611,P769,STORE_21,Completed,2024-06-04,Beauty,Haircare,429.94,7,3009.58,PayPal,Physical,false,38,Male,Silvaview


<h2>Reporting Layer</h2>

</b>For better performance on reporting tables, I've implemented Patition by on Purchase_date and clustering on category,channel,loyalty_program,order_status</b>

In [ ]:
%%bqsql
drop table if exists salesforce-465614.cust_analytics.report_transactions;
CREATE TABLE salesforce-465614.cust_analytics.report_transactions
PARTITION BY Purchase_Date
CLUSTER BY category,channel,loyalty_program,order_status
as
select 
transaction_id,fact.customer_id,fact.product_id,fact.Store_id,order_status,
Purchase_Date,category,sub_category,Unit_Price,quantity,Total_Price,payment_method,store_type as channel,loyalty_program,
age,gender,city from 
(select * from salesforce-465614.cust_analytics.fact_transactions where is_current=True) fact
left join 
(select * from salesforce-465614.cust_analytics.dim_customers where is_current=True)  cust
on fact.customer_id=cust.customer_id
left join 
(select * from salesforce-465614.cust_analytics.dim_products where is_current=True) product 
on fact.product_id=product.product_id
left join 
(select * from salesforce-465614.cust_analytics.dim_stores where is_current=True) stores
on fact.Store_id=stores.Store_id

Query is running:   0%|          |

""


In [55]:
%%bqsql
select min(Purchase_Date) as min_date, max(Purchase_Date) as max_date from salesforce-465614.cust_analytics.report_transactions;


Query is running:   0%|          |

Downloading:   0%|          |

,min_date,max_date
0,2023-09-24,2024-09-23


Optional columns to consider for reporting 
first_name,last_name,signup_date,,email,,state,country

In [ ]:
query = "SELECT * FROM `salesforce-465614.cust_analytics.report_transactions`"

df_transactions = client.query(query).to_dataframe()

c:\Users\Home\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [40]:
df_transactions_1 = df_transactions.drop(columns=['product_id', 'Store_id', 'transaction_id', 'Purchase_Date','city'])

In [54]:
df_transactions_1.head()

,customer_id,order_status,Unit_Price,quantity,Total_Price,payment_method,channel,loyalty_program,age,gender,category_combined
0,C1611,Completed,402.20,4,1608.80,PayPal,Physical,true,49,Female,Electronics_Accessories
1,C1305,Cancelled,949.37,7,6645.59,Credit Card,Online,true,80,Female,Electronics_Accessories
2,C1615,Cancelled,452.15,8,3617.20,Debit Card,Online,true,52,Female,Electronics_Accessories
3,C158,Completed,507.17,3,1521.51,Bank Transfer,Online,true,69,Female,Toys_Board Games
4,C38,Cancelled,507.17,4,2028.68,Bank Transfer,Physical,true,66,Female,Toys_Board Games


In [42]:
df_transactions_1['category_combined'] = df_transactions_1['category'] + '_' + df_transactions_1['sub_category']
df_transactions_1.drop(columns=['category', 'sub_category'], inplace=True)

In [44]:
df_transactions_2 = pd.get_dummies(df_transactions_1, columns=['category_combined', 'channel', 'payment_method', 'order_status'], prefix=['category', 'channel', 'payment_method', 'status'],dtype=int)

In [45]:
df_transactions_2['gender'] = df_transactions_2['gender'].replace({'Female': 0, 'Male': 1})
df_transactions_2['loyalty_program'] = df_transactions_2['loyalty_program'].replace({'false': 0, 'true': 1})
df_transactions_2

C:\Users\Home\AppData\Local\Temp\ipykernel_912\2978431113.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_transactions_2['gender'] = df_transactions_2['gender'].replace({'Female': 0, 'Male': 1})
C:\Users\Home\AppData\Local\Temp\ipykernel_912\2978431113.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_transactions_2['loyalty_program'] = df_transactions_2['loyalty_program'].replace({'false': 0, 'true': 1})


,customer_id,Unit_Price,quantity,Total_Price,loyalty_program,age,gender,category_Beauty_Haircare,category_Beauty_Makeup,category_Beauty_Skincare,...,category_Toys_Educational,channel_Online,channel_Physical,payment_method_Bank Transfer,payment_method_Cash,payment_method_Credit Card,payment_method_Debit Card,payment_method_PayPal,status_Cancelled,status_Completed
0,C1611,402.20,4,1608.80,1,49,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
1,C1305,949.37,7,6645.59,1,80,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
2,C1615,452.15,8,3617.20,1,52,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,C158,507.17,3,1521.51,1,69,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1
4,C38,507.17,4,2028.68,1,66,0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,C776,326.79,3,980.37,1,29,1,0,0,0,...,0,0,1,0,1,0,0,0,0,1
19996,C547,657.95,5,3289.75,1,31,1,0,0,0,...,0,0,1,0,0,0,1,0,0,1
19997,C64,927.10,2,1854.20,1,48,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
19998,C736,767.45,2,1534.90,0,72,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0


In [46]:
customerid = df_transactions_2['customer_id']
df_transactions_3 = df_transactions_2.drop(columns=['customer_id'])

In [48]:
df_transactions_3.describe()

,Unit_Price,quantity,Total_Price,loyalty_program,age,gender,category_Beauty_Haircare,category_Beauty_Makeup,category_Beauty_Skincare,category_Books_Children,...,category_Toys_Educational,channel_Online,channel_Physical,payment_method_Bank Transfer,payment_method_Cash,payment_method_Credit Card,payment_method_Debit Card,payment_method_PayPal,status_Cancelled,status_Completed
count,20000.000000,20000.0,20000.000000,20000.000000,20000.0,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,...,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000
mean,508.455915,5.48555,2795.581718,0.707050,49.09735,0.603250,0.104750,0.038750,0.048450,0.086750,...,0.049200,0.338150,0.661850,0.148200,0.113250,0.265400,0.11050,0.362650,0.328400,0.671600
std,271.109346,2.870854,2231.165574,0.455127,18.349897,0.489236,0.306239,0.193003,0.214721,0.281475,...,0.216291,0.473092,0.473092,0.355307,0.316906,0.441557,0.31352,0.480777,0.469643,0.469643
min,11.540000,1.0,11.540000,0.000000,18.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,241.690000,3.0,904.300000,0.000000,33.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,529.420000,5.0,2246.820000,1.000000,49.0,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000
75%,759.650000,8.0,4293.300000,1.000000,66.0,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.00000,1.000000,1.000000,1.000000
max,997.380000,10.0,9973.800000,1.000000,80.0,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000


In [47]:
pip install scikit-learn --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
